# Refactoring and evolving schema

In [ ]:
#!pip install -r requirements.txt

## Setting up Neo4J database
As in the previous two chapters, we will start by creating a new Neo4j graph database. The steps to do this are detailed here: 

1. Open Neo4j, and choose Add, and then Local DBMS. This will create a new local graph database we are going to work with in this section
2. For this chapter, we will call the new database Refactor DB, and again use the password testpython. Make sure to create a database with these settings, for repeatability. However, you could call this whatever you wish
3. Once the database is created, click on it and choose Start to spin it up for the first time, which may take a minute or so. Once it has started, choose Open to open up the new database in Neo4j Browser.
4. Again, as before, we need to add a new user with administrator privileges, in order to run Cypher on the Neo4j database from Python. In the Neo4j Browser console, run: `:server user add`
5. This will open a window where you can enter username and password for the new user. Here we will use the username admin, and the password  testpython. In the drop down Roles list, choose both admin and PUBLIC. Finally, click Add User.


## Adding constraints

In [ ]:
%%writefile cypher/uniq_user.cql
CREATE CONSTRAINT unique_user
FOR (u:User) 
REQUIRE u.userID IS UNIQUE

In [ ]:
%%writefile cypher/test_user_creation.cql
CREATE (u:User {userID:'100'})

In [ ]:
%%writefile cypher/merge_node_100.cql
MERGE (u:User {userID:'100'})

In [ ]:
%%writefile cypher/clear_env.cql
MATCH (n) DETACH DELETE n

## Pre-change schema

In [ ]:
from graphtastic.database.neo4j import Neo4jConnect
from neo4j import GraphDatabase

In [ ]:
connection = Neo4jConnect('bolt://localhost:7687', 'admin', 'testpython')
print(connection)

In [ ]:
import csv
with open('./data/edges1.csv', 'r') as c:
    reader = csv.reader(c)
    edgelist = [edge for edge in reader]
print(edgelist[:5])


In [ ]:
def add_edge_neo4j(n, m, connection):
    cypher = f'MERGE (u1:User {{userID: "{n}"}}) ' \
             f'MERGE (u2:User {{userID: "{m}"}}) ' \
              'MERGE (u1)-[:FOLLOWS]->(u2)'
    connection.query(cypher)


In [ ]:
connection = Neo4jConnect('bolt://localhost:7687', 'admin', 'testpython')
add_edge_neo4j(edgelist[0][0], edgelist[0][1], connection)
connection.close()

Code to be run in `Neo4j Browser`:

In [ ]:
%%writefile cypher/match_user.cql
MATCH (u:User) RETURN u

In [ ]:
for n, m in edgelist:
    connection = Neo4jConnect('bolt://localhost:7687', 'admin', 'testpython')
    add_edge_neo4j(n, m, connection)
    connection.close()

In [ ]:
with open('./data/circle1.csv', 'r') as c:
    reader = csv.reader(c)
    circles_raw = [row for row in reader]
print(circles_raw)

In [ ]:
def get_user_circle_pairs(circles):
    pairs = []
    for circle in circles:
        circle_pairs = [[user, circle[0]] for user in circle][1:]
        [pairs.append(pair) for pair in circle_pairs]
    return pairs

In [ ]:
pairs = get_user_circle_pairs(circles_raw)

In [ ]:
print(pairs)

In [ ]:
def add_circles_neo4j(user_id, circle_id, connection):
    cypher = f'MATCH (u:User {{userID: "{user_id}"}})' \
         	f'SET u.circle = "{circle_id}"'
    connection.query(cypher)

In [ ]:
for user_id, circle_id in pairs:
	connection = Neo4jConnect('bolt://localhost:7687', 'admin', 'testpython')
	add_circles_neo4j(user_id, circle_id, connection)
	connection.close()

In [ ]:
%%writefile cypher/pairs.cql
MATCH (u:User) RETURN u.circle

## Updating the schema

In [ ]:
%%writefile cypher/find_spec_circle.cql
MATCH (u.User {circle: '5'}) RETURN u

In [ ]:
%%writefile cypher/follower_schema.cql
MATCH (u:User)
WHERE '5' in u.circle RETURN u

In [ ]:
with open('./data/edges2.csv', 'r') as c:
	reader = csv.reader(c)
	edgelist2 = [edge for edge in reader]
 
with open('./data/circle2.csv', 'r') as c:
	reader = csv.reader(c)
	circles2 = [row for row in reader]

In [ ]:
for n, m in edgelist2:
	connection = Neo4jConnect('bolt://localhost:7687', 'admin', 'testpython')
	add_edge_neo4j(n, m, connection)
	connection.close()

In [ ]:
pairs2 = get_user_circle_pairs(circles2)

In [ ]:
def add_circles_neo4j_new_schema(user_id, circle_id, connection):
	cypher = f'MATCH (u:User {{userID: "{user_id}"}})' \
         	f'MERGE (c:Circle {{circleID: "{circle_id}"}})' \
          	'MERGE (u)-[:MEMBER_OF]->(c)'
	connection.query(cypher)

In [ ]:
for user_id, circle_id in pairs2:
    connection = Neo4jConnect('bolt://localhost:7687', 'admin', 'testpython')
    add_circles_neo4j_new_schema(user_id, circle_id, connection)
    connection.close()

In [ ]:
%%writefile cypher/member_of_circles.cql
MATCH (u:User)-[:MEMBER_OF]->(c:Circle {circleID: '5'})
RETURN u, c

In [ ]:
%%writefile cypher/remove_the_old_data.cql
MATCH (u:User)
REMOVE u.circle